In [2]:
import pandas as pd
import re
import openpyxl
from IPython.display import display
import numpy as np

In [16]:
from dotenv import load_dotenv
from os import getenv
from openai import OpenAI
import secrets
import string

# Load environment variables from .env file
load_dotenv("../../.env")


class chatGPT():
    def __init__(self):
        # Get API key from environment variable
        my_key = getenv("OPENAI_API_KEY")
        if my_key is None:
            raise ValueError("API key not found in .env file")
        
        # Initialize OpenAI client with API key
        self.client = OpenAI(api_key=my_key)
        
    def ask(self, q):
        stream = self.client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": q}],
            stream=True,
            temperature=1
        )
        response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                response += chunk.choices[0].delta.content

        self.response = response
        return response

# Example usage
chatbot = chatGPT()
response = chatbot.ask("Hello, how are you?")
print(response)


As an artificial intelligence, I don't possess feelings but thanks for asking. How can I assist you today?


In [17]:

class ExcelHandler:
    def __init__(self):
        self.workbook = self.getWorkbook()

    def getWorkbook(self, filename='../output/tasks.xlsx'):
        try:
            workbook = openpyxl.load_workbook(filename)
        except FileNotFoundError:
            workbook = openpyxl.Workbook()
        return workbook

    def createWorksheet(self, sheet_name):
        if sheet_name in self.workbook.sheetnames:
            worksheet = self.workbook[sheet_name]
            self.workbook.remove(worksheet) 
        print(sheet_name)
        self.workbook.create_sheet(title=sheet_name)
        bold_font = openpyxl.styles.Font(bold=True)
        worksheet = self.workbook[sheet_name]   
        for i, column in enumerate(worksheet.iter_cols(max_row=1, max_col=6)):
            worksheet.column_dimensions[column[0].column_letter].width = 40
        self.workbook.save('../output/tasks.xlsx')

    def getWorksheet(self, sheet_name):
        worksheet = self.workbook[sheet_name]
        return worksheet

    def writeDfToExcel(self, sheet_name, df, start_row=3):
        worksheet = self.getWorksheet(sheet_name)
        for index, row in df.iterrows():
            row_index = start_row + index
            worksheet.cell(row=row_index, column=2, value=row['Prompt']).alignment = openpyxl.styles.Alignment(wrap_text=True)
            worksheet.cell(row=row_index, column=3, value=row['Response']).alignment = openpyxl.styles.Alignment(wrap_text=True)
        self.workbook.save('../output/tasks.xlsx')
        
    def writeRowToExcel(self, sheet_name, start_row, start_col, values, bold=False):
        print("\n")
        print(values)
        print("\n")
        
        worksheet = self.getWorksheet(sheet_name)
        for i, val in enumerate(values):
            cell = worksheet.cell(row=start_row, column=i+start_col)
            cell.value = val
            cell.alignment = openpyxl.styles.Alignment(wrap_text=True)
            if bold:
                cell.font = openpyxl.styles.Font(bold=True)
        
    def save(self):  
        self.workbook.save('../output/tasks.xlsx')

    def displayWorksheet(self, sheet_name):
        worksheet = self.getWorksheet(sheet_name)
        data = worksheet.values
        columns = next(data)
        df = pd.DataFrame(data, columns=columns)
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        pd.set_option('display.max_colwidth', None)
        display(df)


In [18]:
class GovOccupations:
    def getGovTask(self, occupation):
        self.getGovData()
        occ_tasks = self.allOccupations.loc[self.allOccupations.Title == occupation]
        gov_tasks = list(occ_tasks.Task)
        return gov_tasks
    def getGovData(self):
        self.allOccupations = pd.read_csv("../input/Task Statements.csv")
        return self.allOccupations

In [19]:
class Prompting:
    def __init__(self):
        excel_file_path = '../input/prompts.xlsx'
        self.prompts_df = pd.read_excel(excel_file_path)
        self.chat = chatGPT()
        self.promptChat = chatGPT()
        self.excel_handler = ExcelHandler()
        
        self.occupation = "Computer Science Teachers, Postsecondary"
        self.gov_tasks = GovOccupations().getGovTask(self.occupation)
        if (len(self.gov_tasks) > 0):
            self.chatGPTPrompting(self.occupation, self.gov_tasks)
        
    def splitResponse(self, response):
        splits = re.split(r'\d+\.\s', response)
        splits = [s.strip("\n") for s in splits if s]
        return splits

    def fixPrompt(self,prompt):
        prompt = f"This is my prompt: \" {prompt} \" \n\n This is what I need you to do: Fix wording in prompt, Do not change meaning, Do not add words, DO NOT ANSWER PROMPT"
        print(prompt+"\n")
        prompt = self.promptChat.ask(prompt)
        return prompt
        
    def getPrompt(self, i, *vars):
        prompt = self.prompts_df.Prompt[i - 1]
        specs = self.prompts_df.Specifications[i - 1]
        print(specs)
        prompt = re.sub(r'\{.*?\}', '{}', prompt)
        prompt = prompt.format(*vars)
        print(f"Before {prompt}\n")
        prompt = self.fixPrompt(prompt)
        if not pd.isnull(specs):
            prompt = prompt + " " + specs
        print(f"After {prompt}\n")
        return prompt

    def display(self):
        sheet_name = self.occupation[:31]
        self.excel_handler.displayWorksheet(sheet_name)

        
    def chatGPTPrompting(self, occupation, tasks):
        sheet_name = occupation[:31]
        row = 1
        self.excel_handler.createWorksheet(sheet_name)
        self.excel_handler.writeRowToExcel(sheet_name, row,1,["Task",self.occupation],bold=True)
        row += 1
        
        for task_number, task in list(enumerate(tasks, start=1)):

            self.excel_handler.writeRowToExcel(sheet_name, row,1,[task])
            row += 1
            
            prompt = self.getPrompt(1, occupation, task) 
            response = self.chat.ask(prompt)
            self.excel_handler.writeRowToExcel(sheet_name, row,2,[prompt,response])
            row += 1
            

            self.excel_handler.writeRowToExcel(sheet_name, row,2,["Go through all situations:", "Prompt", "Response"],bold=True)
            row += 1
            self.excel_handler.save()
            
            situations = self.splitResponse(response)
            self.promptChat = chatGPT()
            for situation in situations[:1]:

                prompt = self.getPrompt(2, occupation, task) 
                response = self.chat.ask(prompt)
                self.excel_handler.writeRowToExcel(sheet_name, row,3,[prompt,response])
                row += 1
                
                self.excel_handler.writeRowToExcel(sheet_name, row,3,["Go through all subtaks:", "Prompt", "Response"],bold=True)
                row += 1
                self.excel_handler.save()
                
                subtasks = self.splitResponse(response)
                self.promptChat = chatGPT()
                for subtask in subtasks[:1]:
                    prompt = self.getPrompt(3, occupation, situation, subtask)
                    response = self.chat.ask(prompt)
                    self.excel_handler.writeRowToExcel(sheet_name, row,4,[prompt,response])
                    row += 1

                    self.excel_handler.writeRowToExcel(sheet_name, row,4,["Go through all outputs:", "Prompt", "Response"],bold=True)
                    row += 1
                    outputs = self.splitResponse(response)
                    for output in outputs[:1]:
                        prompt = self.getPrompt(4, occupation, situation, subtask, output)
                        response = self.chat.ask(prompt)
                        self.excel_handler.writeRowToExcel(sheet_name, row,5,[prompt,response])
                        row += 1
        
                        self.excel_handler.save()

x = Prompting()


Computer Science Teachers, Post


['Task', 'Computer Science Teachers, Postsecondary']




['Prepare course materials, such as syllabi, homework assignments, and handouts.']


Respond in numbered list with max 3 bullet points with max 15 words.
Before I am an Computer Science Teachers, Postsecondary and I have to Prepare course materials, such as syllabi, homework assignments, and handouts.. Give a company/client/case/project I might have.

This is my prompt: " I am an Computer Science Teachers, Postsecondary and I have to Prepare course materials, such as syllabi, homework assignments, and handouts.. Give a company/client/case/project I might have. " 

 This is what I need you to do: Fix wording in prompt, Do not change meaning, Do not add words, DO NOT ANSWER PROMPT

After I am a Postsecondary Computer Science Teacher tasked with preparing course materials, including syllabi, homework assignments, and handouts. Supply a company/client/case/project I might oversee. Respond in numbered

KeyboardInterrupt: 

In [ ]:
x.display()

In [107]:


titles_df = pd.DataFrame(GovOccupations().getGovData().Title.unique(), columns=['Title'])
titles_df.to_csv('../input/Occupations.csv', index=False)
titles_df.head()

,Title
0,Chief Executives
1,Chief Sustainability Officers
2,General and Operations Managers
3,Legislators
4,Advertising and Promotions Managers
